# Drawing NN step by step

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display, HTML

#### helper functions for drawing
From DeepDream tensorflow tutorial: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb

In [28]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def

def show_graph(graph_def=tf.get_default_graph(), max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    


### Step 1. Placeholders

In [3]:
X = tf.placeholder("float32", name="X")
Y = tf.placeholder("float32", name="Y")

In [4]:
# show_graph()

### Step 2. Variables

In [5]:
w1 = tf.Variable(tf.random_normal([1, 10]), dtype="float32", name="w1")
b1 = tf.Variable(tf.constant(1.), dtype="float32", name="b1")

w2 = tf.Variable(tf.random_normal([10, 1]), dtype="float32", name="w2")
b2 = tf.Variable(tf.constant(1.), dtype="float32", name="b2")

In [6]:
# show_graph()

### Step 3. Connecting

In [7]:
activation = tf.nn.tanh
l1 = tf.matmul(X, w1) + b1

In [8]:
# show_graph()

In [9]:
l1 = activation(l1)

In [10]:
# show_graph()

In [11]:
l2 = activation(tf.matmul(l1, w2) + b2)

In [17]:
# show_graph()

In [15]:
output = l2
cost = tf.reduce_mean(abs(output - Y)**2)

In [18]:
# show_graph()

In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cost)

In [29]:
show_graph()